# KPA User Data Management & Analysis

This notebook provides tools for:
- 🔧 Testing KPA API connectivity
- 👥 Fetching and analyzing user data  
- 📸 Testing photo integration
- 📊 Data exploration and filtering
- 📋 Export utilities for raffle system

**Updated**: August 25, 2025 - After KPA integration optimization

In [1]:
# Import required libraries
import os
import requests
import pandas as pd
import json
from typing import Dict, List, Optional
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set display options for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("📦 Libraries imported successfully!")
print(f"🕒 Notebook initialized at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

📦 Libraries imported successfully!
🕒 Notebook initialized at: 2025-08-25 21:17:23


In [4]:
# KPA API Configuration
API_BASE = "https://api.kpaehs.com/v1"

# Load KPA token from environment (try both variable names)
KPA_TOKEN = os.getenv("KPA_TOKEN") or os.getenv("KPA_API_TOKEN")

# Display configuration status
print("🔧 KPA API Configuration")
print("=" * 40)
print(f"API Base URL: {API_BASE}")
print(f"Token Status: {'✅ Found' if KPA_TOKEN else '❌ Missing'}")

if KPA_TOKEN:
    print(f"Token Preview: {KPA_TOKEN[:20]}...")
    print("🎯 Ready to test KPA connectivity!")
else:
    print("⚠️  KPA_TOKEN or KPA_API_TOKEN environment variable not found")
    print("💡 Make sure your .env file or secrets are configured properly")

🔧 KPA API Configuration
API Base URL: https://api.kpaehs.com/v1
Token Status: ✅ Found
Token Preview: pTfES8COPXiB3fCCE0ud...
🎯 Ready to test KPA connectivity!


In [2]:
# Load environment variables from secrets folder
from dotenv import load_dotenv

# Try to load from secrets folder
env_path = '/workspaces/raffle-randomizer/secrets/.env'
if os.path.exists(env_path):
    load_dotenv(env_path)
    print(f"🔑 Loaded environment from: {env_path}")
else:
    print("⚠️ No .env file found in secrets folder")

# Also try loading from current directory
load_dotenv()
print("🔍 Environment loading complete")

🔑 Loaded environment from: /workspaces/raffle-randomizer/secrets/.env
🔍 Environment loading complete


In [5]:
# Test KPA API Connectivity
def test_kpa_integration():
    """Test KPA API connectivity and available endpoints"""
    print("🔍 Testing KPA API Integration")
    print("=" * 50)
    
    if not KPA_TOKEN:
        print("❌ No KPA token available - cannot test")
        return None
    
    # Test basic connectivity with users.list
    try:
        print("📡 Testing users.list endpoint...")
        data = {"token": KPA_TOKEN}
        response = requests.post(f"{API_BASE}/users.list", json=data, timeout=10)
        
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            result = response.json()
            print("✅ KPA API connection successful!")
            
            if isinstance(result, list):
                print(f"📊 Found {len(result)} users")
                if result:
                    print("🔍 Sample user fields:")
                    sample_user = result[0]
                    for key in sorted(sample_user.keys()):
                        print(f"  • {key}")
                return result
            else:
                print(f"📋 Response type: {type(result)}")
                print(f"📋 Response keys: {list(result.keys()) if isinstance(result, dict) else 'Not a dict'}")
                return result
        else:
            print(f"❌ API Error: {response.status_code}")
            try:
                error_data = response.json()
                print(f"Error details: {error_data}")
            except:
                print(f"Raw response: {response.text}")
            return None
            
    except Exception as e:
        print(f"❌ Connection failed: {str(e)}")
        return None

# Run the test
kpa_data = test_kpa_integration()

🔍 Testing KPA API Integration
📡 Testing users.list endpoint...
Status Code: 200
✅ KPA API connection successful!
📋 Response type: <class 'dict'>
📋 Response keys: ['users', 'columns', 'ok']
Status Code: 200
✅ KPA API connection successful!
📋 Response type: <class 'dict'>
📋 Response keys: ['users', 'columns', 'ok']


In [ ]:
# Analyze User Data
print("📊 KPA API Response Analysis")
print("=" * 40)

if kpa_data:
    print(f"📋 Response Type: {type(kpa_data)}")
    print(f"📋 Response Keys: {list(kpa_data.keys()) if isinstance(kpa_data, dict) else 'Not a dict'}")
    
    # Check if users are in a nested structure
    if isinstance(kpa_data, dict) and 'users' in kpa_data:
        users_list = kpa_data['users']
        print(f"? Found users list with {len(users_list)} users")
        
        if users_list and len(users_list) > 0:
            # Convert to DataFrame for easier analysis
            df = pd.DataFrame(users_list)
            
            print(f"📋 Available Fields: {len(df.columns)}")
            print("\n🔍 Data Overview:")
            print(df.info())
            
            print("\n📈 Column Summary:")
            for col in df.columns:
                non_null_count = df[col].notna().sum()
                print(f"  • {col}: {non_null_count}/{len(df)} ({non_null_count/len(df)*100:.1f}%)")
            
            # Look for photo fields
            photo_fields = [col for col in df.columns if 'photo' in col.lower() or 'image' in col.lower() or 'picture' in col.lower()]
            if photo_fields:
                print(f"\n📸 Photo Fields Found: {photo_fields}")
                for field in photo_fields:
                    photo_count = df[field].notna().sum()
                    print(f"  • {field}: {photo_count} users have photos")
            else:
                print("\n📸 No obvious photo fields found")
                print("🔍 Fields containing 'http' or URLs:")
                for col in df.columns:
                    if df[col].dtype == 'object':
                        url_count = df[col].astype(str).str.contains('http', na=False).sum()
                        if url_count > 0:
                            print(f"  • {col}: {url_count} entries contain 'http'")
        else:
            print("❌ Users list is empty")
    elif isinstance(kpa_data, list):
        print("📊 Data appears to be a direct list")
        df = pd.DataFrame(kpa_data)
        print(f"👥 Total Users: {len(df)}")
        print(f"📋 Available Fields: {len(df.columns)}")
    else:
        print("❌ Unexpected data structure")
        print(f"Data content: {kpa_data}")
else:
    print("❌ No user data available for analysis")

In [ ]:
# Test KPA API with corrected parameter name
def test_users_info_endpoint():
    """Test the users.info endpoint with correct 'id' parameter"""
    print("🔧 Testing KPA users.info with corrected parameter")
    print("=" * 50)
    
    if not KPA_TOKEN:
        print("❌ No KPA token available")
        return None
    
    # Test with a sample user ID (using your example format)
    test_id = "6470af6754bfbc1d5ce9bb61"  # Your example ID
    
    try:
        # Test with corrected parameter name: "id" instead of "user_id"
        data = {"token": KPA_TOKEN, "id": test_id}
        response = requests.post(f"{API_BASE}/users.info", json=data, timeout=10)
        
        print(f"📡 Testing with ID: {test_id}")
        print(f"📋 Status Code: {response.status_code}")
        
        if response.status_code == 200:
            result = response.json()
            print("✅ users.info endpoint successful!")
            print(f"📊 Response type: {type(result)}")
            print(f"📋 Response keys: {list(result.keys()) if isinstance(result, dict) else 'Not a dict'}")
            
            # Look for user data
            if isinstance(result, dict):
                if 'user' in result:
                    user_data = result['user']
                    print(f"👤 User data found: {type(user_data)}")
                    if isinstance(user_data, dict):
                        print(f"📋 User fields: {list(user_data.keys())}")
                elif 'ok' in result:
                    print(f"🔍 API response OK: {result.get('ok')}")
                    
            return result
        else:
            print(f"❌ API Error: {response.status_code}")
            try:
                error_data = response.json()
                print(f"💥 Error details: {error_data}")
            except:
                print(f"📄 Raw response: {response.text}")
            return None
            
    except Exception as e:
        print(f"❌ Request failed: {str(e)}")
        return None

# Run the test
test_result = test_users_info_endpoint()

In [ ]:
# Quick check of kpa_data structure
print("🔍 Quick Data Check")
print("=" * 30)
print(f"Type: {type(kpa_data)}")
if isinstance(kpa_data, dict):
    print(f"Keys: {list(kpa_data.keys())}")
    if 'users' in kpa_data:
        print(f"Users count: {len(kpa_data['users'])}")
        if kpa_data['users']:
            print(f"First user keys: {list(kpa_data['users'][0].keys())}")
elif isinstance(kpa_data, list):
    print(f"List length: {len(kpa_data)}")
else:
    print(f"Data: {kpa_data}")

In [ ]:
# Test Photo Integration
def test_photo_loading(sample_size=3):
    """Test photo loading for a sample of users"""
    if not kpa_data:
        print("❌ No user data available for photo testing")
        return
    
    print("📸 Testing Photo Loading")
    print("=" * 40)
    
    # Handle both list and dict structures
    users_list = None
    if isinstance(kpa_data, dict) and 'users' in kpa_data:
        users_list = kpa_data['users']
        print(f"📊 Using users from API response ({len(users_list)} users)")
    elif isinstance(kpa_data, list):
        users_list = kpa_data
        print(f"📊 Using direct user list ({len(users_list)} users)")
    else:
        print("❌ Unexpected data structure for photo testing")
        return
    
    if not users_list:
        print("❌ No users found for photo testing")
        return
    
    df = pd.DataFrame(users_list)
    
    # Find potential photo fields
    photo_candidates = []
    for col in df.columns:
        if df[col].dtype == 'object':
            url_count = df[col].astype(str).str.contains('http', na=False).sum()
            if url_count > 0:
                photo_candidates.append((col, url_count))
    
    if not photo_candidates:
        print("❌ No URL fields found for photo testing")
        return
    
    print(f"🔍 Found {len(photo_candidates)} potential photo fields:")
    for field, count in photo_candidates:
        print(f"  • {field}: {count} URLs")
    
    # Test the most promising field
    best_field = max(photo_candidates, key=lambda x: x[1])[0]
    print(f"\n🎯 Testing field: {best_field}")
    
    # Get sample URLs
    sample_urls = df[df[best_field].notna()][best_field].head(sample_size)
    
    for i, url in enumerate(sample_urls, 1):
        print(f"\n📷 Testing photo {i}: {str(url)[:60]}...")
        try:
            response = requests.get(str(url), timeout=10)
            if response.status_code == 200:
                content_type = response.headers.get('content-type', '')
                size = len(response.content)
                print(f"  ✅ Success! Type: {content_type}, Size: {size:,} bytes")
            else:
                print(f"  ❌ Failed: HTTP {response.status_code}")
        except Exception as e:
            print(f"  ❌ Error: {str(e)}")

# Run photo test
test_photo_loading()

In [ ]:
# Data Exploration & Filtering Tools
def explore_user_data():
    """Interactive exploration of user data"""
    if not kpa_data:
        print("❌ No user data available")
        return
    
    print("🔍 User Data Explorer")
    print("=" * 40)
    
    # Handle both list and dict structures
    users_list = None
    if isinstance(kpa_data, dict) and 'users' in kpa_data:
        users_list = kpa_data['users']
        print(f"📊 Using users from API response ({len(users_list)} users)")
    elif isinstance(kpa_data, list):
        users_list = kpa_data
        print(f"📊 Using direct user list ({len(users_list)} users)")
    else:
        print("❌ Unexpected data structure")
        return
    
    if not users_list:
        print("❌ No users found")
        return
    
    df = pd.DataFrame(users_list)
    
    # Show sample records
    print("\n📋 Sample Records (first 3):")
    print(df.head(3).to_string())
    
    # Look for common fields useful for raffles
    important_fields = ['name', 'first_name', 'last_name', 'email', 'department', 
                       'location', 'position', 'level', 'status', 'active']
    
    found_fields = [field for field in important_fields if field in df.columns]
    print(f"\n🎯 Important Fields Found: {found_fields}")
    
    # Check for location/department data
    location_fields = [col for col in df.columns if any(word in col.lower() 
                      for word in ['location', 'site', 'office', 'building'])]
    if location_fields:
        print(f"\n📍 Location Fields: {location_fields}")
        for field in location_fields:
            unique_values = df[field].nunique()
            print(f"  • {field}: {unique_values} unique values")
            if unique_values <= 10:
                print(f"    Values: {list(df[field].unique())}")
    
    # Check for level/department data
    dept_fields = [col for col in df.columns if any(word in col.lower() 
                  for word in ['department', 'dept', 'division', 'team', 'level'])]
    if dept_fields:
        print(f"\n🏢 Department/Level Fields: {dept_fields}")
        for field in dept_fields:
            unique_values = df[field].nunique()
            print(f"  • {field}: {unique_values} unique values")
            if unique_values <= 15:
                print(f"    Values: {list(df[field].unique())}")

# Run exploration
explore_user_data()

In [ ]:
# Export Utilities for Raffle System
def create_raffle_export():
    """Create CSV export optimized for the raffle system"""
    if not kpa_data:
        print("❌ No user data available for export")
        return
    
    print("📋 Creating Raffle System Export")
    print("=" * 40)
    
    # Handle both list and dict structures
    users_list = None
    if isinstance(kpa_data, dict) and 'users' in kpa_data:
        users_list = kpa_data['users']
        print(f"📊 Using users from API response ({len(users_list)} users)")
    elif isinstance(kpa_data, list):
        users_list = kpa_data
        print(f"? Using direct user list ({len(users_list)} users)")
    else:
        print("❌ Unexpected data structure for export")
        return
    
    if not users_list:
        print("❌ No users found for export")
        return
    
    df = pd.DataFrame(users_list)
    
    # Map fields to raffle system requirements
    field_mapping = {}
    
    # Try to find name fields
    name_candidates = [col for col in df.columns if 'name' in col.lower()]
    if name_candidates:
        # Prefer full name, then first + last
        if any('full' in col.lower() for col in name_candidates):
            field_mapping['Name'] = next(col for col in name_candidates if 'full' in col.lower())
        elif 'name' in df.columns:
            field_mapping['Name'] = 'name'
        else:
            field_mapping['Name'] = name_candidates[0]
    
    # Try to find location
    location_candidates = [col for col in df.columns if any(word in col.lower() 
                          for word in ['location', 'site', 'office', 'building'])]
    if location_candidates:
        field_mapping['Location'] = location_candidates[0]
    
    # Try to find level/department
    level_candidates = [col for col in df.columns if any(word in col.lower() 
                       for word in ['level', 'grade', 'position', 'title'])]
    if level_candidates:
        field_mapping['Level'] = level_candidates[0]
    
    # Try to find photo field
    photo_candidates = []
    for col in df.columns:
        if df[col].dtype == 'object':
            url_count = df[col].astype(str).str.contains('http', na=False).sum()
            if url_count > 0:
                photo_candidates.append((col, url_count))
    
    if photo_candidates:
        best_photo_field = max(photo_candidates, key=lambda x: x[1])[0]
        field_mapping['Photo'] = best_photo_field
    
    print("\n🎯 Field Mapping:")
    for raffle_field, kpa_field in field_mapping.items():
        print(f"  • {raffle_field}: {kpa_field}")
    
    # Create export DataFrame
    export_df = pd.DataFrame()
    for raffle_field, kpa_field in field_mapping.items():
        if kpa_field in df.columns:
            export_df[raffle_field] = df[kpa_field]
    
    # Add any missing required columns
    required_cols = ['Name', 'Location', 'Level']
    for col in required_cols:
        if col not in export_df.columns:
            export_df[col] = 'Unknown'
    
    # Filter out users without names
    if 'Name' in export_df.columns:
        export_df = export_df[export_df['Name'].notna() & (export_df['Name'] != '')]
    
    print(f"\n📊 Export Summary:")
    print(f"  • Total records: {len(export_df)}")
    print(f"  • Columns: {list(export_df.columns)}")
    
    # Show sample
    print(f"\n📋 Sample Export Data:")
    print(export_df.head().to_string())
    
    # Save to CSV
    filename = f"kpa_raffle_export_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    export_df.to_csv(filename, index=False)
    print(f"\n💾 Exported to: {filename}")
    
    return export_df

# Create export
raffle_export = create_raffle_export()

## 📊 Summary & Next Steps

This notebook provides a comprehensive toolkit for managing KPA user data:

### ✅ What This Notebook Does:
1. **🔧 Tests KPA API connectivity** - Verifies token and endpoint access
2. **📊 Analyzes user data structure** - Shows available fields and data quality
3. **📸 Tests photo integration** - Validates photo URLs work properly
4. **🔍 Explores data** - Helps understand location, department, and level data
5. **📋 Creates raffle exports** - Generates CSV files optimized for the raffle system

### 🎯 Recommended Workflow:
1. Run all cells from top to bottom
2. Review the data analysis to understand your user base
3. Use the export function to create raffle-ready CSV files
4. Test photo loading to ensure images work in the raffle system

### 💡 Tips for Raffle System:
- The exported CSV can be directly uploaded to your raffle app
- Photo URLs are tested to ensure they work reliably
- Location and level data helps with targeted raffles
- Regular exports help keep your raffle data current

In [ ]:
# Simple kernel test
print("🧪 Kernel Test")
print(f"KPA data type: {type(kpa_data)}")
if isinstance(kpa_data, dict):
    print(f"Keys: {list(kpa_data.keys())}")
print("✅ Kernel is responsive!")